<a href="https://colab.research.google.com/github/agarwalsourabh55/Deep-learning/blob/NewsCategoryClassifier/NewsCategoryDeepLearning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import torch
#a variable sourcefile is used to store the argument that is passed while running the script. Sys.argv is the argument taking the file name while executing the script and 1 indicates the argument number while 0 being the script that we are executing.
sourcefile = "/content/gdrive/MyDrive/News_Category_JSON.json"
data =  pd.read_json(sourcefile,lines= True)

In [3]:
data

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [4]:
## Lts See how many categories are present
len(data['category'].unique())

41

In [5]:
## Defining Tokenizer for the Embeddings
## For this use case trying hands on with bert tokenizer

!pip install transformers
from transformers import BertTokenizer
text = data['headline'].tolist()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoding = tokenizer.encode_plus(text[0],
                                 add_special_tokens = True,
                                 truncation = True,
                                 padding = "max_length",
                                 return_attention_mask = True,
                                 return_tensors = "pt")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.1 MB/s 
     |████████████████████████████████| 101 kB 6.0 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 6.6 MB 62.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import random
## labelencoding the target column
labels = dict(zip(data['category'].unique(), range(1,data['category'].nunique()+1)))
labels

{'ARTS': 31,
 'ARTS & CULTURE': 22,
 'BLACK VOICES': 7,
 'BUSINESS': 12,
 'COLLEGE': 20,
 'COMEDY': 9,
 'CRIME': 1,
 'CULTURE & ARTS': 41,
 'DIVORCE': 36,
 'EDUCATION': 19,
 'ENTERTAINMENT': 2,
 'ENVIRONMENT': 40,
 'FIFTY': 30,
 'FOOD & DRINK': 38,
 'GOOD NEWS': 28,
 'GREEN': 24,
 'HEALTHY LIVING': 26,
 'HOME & LIVING': 34,
 'IMPACT': 4,
 'LATINO VOICES': 18,
 'MEDIA': 14,
 'MONEY': 39,
 'PARENTING': 33,
 'PARENTS': 21,
 'POLITICS': 5,
 'QUEER VOICES': 10,
 'RELIGION': 16,
 'SCIENCE': 17,
 'SPORTS': 11,
 'STYLE': 23,
 'STYLE & BEAUTY': 35,
 'TASTE': 25,
 'TECH': 15,
 'THE WORLDPOST': 27,
 'TRAVEL': 13,
 'WEDDINGS': 37,
 'WEIRD NEWS': 6,
 'WELLNESS': 32,
 'WOMEN': 8,
 'WORLD NEWS': 3,
 'WORLDPOST': 29}

In [7]:
## Defining Dataloaders class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df,target_column):

        self.labels = [labels[label] for label in df[target_column]]
        self.texts = [tokenizer(text,
                                padding='max_length',
                                max_length = 512,
                                truncation=True,
                                return_tensors="pt") for text in df['headline']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)
        return batch_texts, batch_y


In [8]:
from sklearn.model_selection import train_test_split
# y= data['category']
# x = data.drop('category',axis=1)
train_data, val_data  = train_test_split(data,
                                         test_size = 0.3)

train, val = Dataset(train_data,'category'), Dataset(val_data,'category')

train_dataloader = torch.utils.data.DataLoader(train,
                                               batch_size=2,
                                               shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val,
                                             batch_size=2)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 41)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

In [13]:
from torch.optim import Adam
from tqdm import tqdm
from torch import nn
from transformers import BertModel
import numpy as np


EPOCHS = 5
model = BertClassifier()
LR = 1e-6

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr= LR)
if use_cuda:
      model = model.cuda()
      criterion = criterion.cuda()

epochs = 5
for epoch_num in range(epochs):
      total_acc_train = 0
      total_loss_train = 0
      for train_input, train_label in tqdm(train_dataloader):
          train_label = train_label.to(device)
          mask = train_input['attention_mask'].to(device)
          input_id = train_input['input_ids'].squeeze(1).to(device)
          print(input_id)
          output = model(input_id, mask)
          
          batch_loss = criterion(output, train_label.long())
          total_loss_train += batch_loss.item()
          
          acc = (output.argmax(dim=1) == train_label).sum().item()
          total_acc_train += acc

          model.zero_grad()
          batch_loss.backward()
          optimizer.step()
      
      total_acc_val = 0
      total_loss_val = 0

      with torch.no_grad():

          for val_input, val_label in val_dataloader:

              val_label = val_label.to(device)
              mask = val_input['attention_mask'].to(device)
              input_id = val_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              batch_loss = criterion(output, val_label.long())
              total_loss_val += batch_loss.item()
              
              acc = (output.argmax(dim=1) == val_label).sum().item()
              total_acc_val += acc
      
      print(
          f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
          | Train Accuracy: {total_acc_train / len(train_data): .3f} \
          | Val Loss: {total_loss_val / len(val_data): .3f} \
          | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/70299 [00:00<?, ?it/s]

tensor([[  101, 20070,  2003,  ...,     0,     0,     0],
        [  101,  4257,  5308,  ...,     0,     0,     0]])


  0%|          | 1/70299 [00:12<245:48:12, 12.59s/it]

tensor([[ 101, 1996, 4274,  ...,    0,    0,    0],
        [ 101, 9795, 2375,  ...,    0,    0,    0]])


  0%|          | 2/70299 [00:24<233:10:20, 11.94s/it]

tensor([[ 101, 5088, 2732,  ...,    0,    0,    0],
        [ 101, 2054, 2106,  ...,    0,    0,    0]])


  0%|          | 3/70299 [00:36<241:43:44, 12.38s/it]

tensor([[ 101, 1996, 2880,  ...,    0,    0,    0],
        [ 101, 2122, 5637,  ...,    0,    0,    0]])


  0%|          | 3/70299 [00:37<242:46:42, 12.43s/it]


KeyboardInterrupt: ignored